In [58]:
from os import listdir
import os
import pickle

done = 0
data = {}
for f in listdir('data'):
    data[f[:-2]] = pickle.load(open('data/' + f, 'rb'), encoding='latin1')


In [59]:
for i in data:
    print(len(data[i]))

153
735
222
148
108
1847
189
63


In [55]:
import tensorflow as tf
from tensorflow.nn.rnn_cell import LSTMCell
import tensorflow.layers

class LSTM:
    def __init__(self):
        rnn_layers = [tf.nn.rnn_cell.LSTMCell(x) for x in [128, 128, 1]]
        self.lstm = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
        self.inp = tf.placeholder(tf.uint8, shape=[None, 20], name='in')
        self.data = tf.one_hot(self.inp, depth=4)
        self.labels = tf.placeholder(tf.float32, shape=[None, 1], name='labels')
        self.results, _ = tf.nn.dynamic_rnn(self.lstm, self.data, dtype=tf.float32)
        self.loss = tf.reduce_mean(tf.squared_difference(self.results, self.labels))
        self.op = tf.train.AdamOptimizer(0.001).minimize(self.loss)
    def train(self, sess, seq, labels):
        loss, _ = sess.run([self.loss, self.op], 
                           feed_dict={self.inp: seq, self.labels: labels})
        return loss
    def evaluate(self, sess, seq, labels):
        loss = sess.run(self.loss, 
                           feed_dict={self.inp: seq, self.labels: labels})
        return loss
    def run(self, sess, seq):
        results = sess.run(self.results, 
                           feed_dict={self.inp: seq})
        return results  

In [56]:
val_p = 0.5
val_n = 0.0

pA=[('A' * 20, val_p)],
pB=[('T' * 20, val_p)],
pC=[('C' * 20, val_p)],
pD=[('G' * 20, val_p)],
nA=[('A' * 20, val_n)],
nB=[('T' * 20, val_n)],
nC=[('C' * 20, val_n)],
nD=[('G' * 20, val_n)],

toy_env = dict(
    tA = 5 * nA + 5 * nB + 10 * pA,
    tB = 5 * pC + 5 * pB + 10 * nD,
    tC = 5 * pA + 5 * pD + 5 * nC + 5 * nB,
    tD = 5 * nA + 5 * nB + 10 * pA,
    tE = 5 * pC + 5 * pB + 10 * nD,
    vA = 5 * pA + 5 * pD + 5 * nC + 5 * nB,
    vB = 5 * pA + 5 * pD + 5 * nC + 5 * nB,
    vC = 5 * pC + 5 * pB + 10 * nD,
)
data = {}
for x in toy_env:
    data[x] = [i[0] for i in toy_env[x]]


In [57]:
genes = [x for x in data]
train = genes[:5]
validate = genes[5:7]

In [50]:
tf.reset_default_graph()
sess = tf.Session()
net = LSTM()
sess.run(tf.global_variables_initializer())

In [51]:
def vec(s):
    return np.array(['ATCG'.index(x) for x in s])

In [52]:
from tqdm import tqdm as tqdm
from random import *
import numpy as np
def training():
    loss = []
    pbar = tqdm(list(range(2000)))
    for i in pbar:
        train_data = data[choice(train)]
        shuffle(train_data)
        sample = train_data[:20]
        values = np.array([vec(x[0]) for x in sample])
        labels = np.array([[x[1]] for x in sample])
        loss.append(net.train(sess, values, labels))

    return np.mean(np.array(loss))

In [53]:
def validation():
    loss = []
    pbar = tqdm(list(range(500)))
    for i in pbar:
        train_data = data[choice(validate)]
        shuffle(train_data)
        sample = train_data[:20]
        values = np.array([vec(x[0]) for x in sample])
        labels = np.array([[x[1]] for x in sample])
        loss.append(net.evaluate(sess, values, labels))

    return np.mean(np.array(loss))

In [54]:
import matplotlib.pyplot as plt
loss_t = []
loss_v = []
for i in range(6):
    loss_t += [training()]
    loss_v += [validation()]

plt.title('Loss')
plt.plot(np.array(loss_t), label='training')
plt.plot(np.array(loss_v), label='validation')
plt.legend()
plt.show()









  0%|          | 0/2000 [00:00<?, ?it/s]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [2, 2, 

ValueError: setting an array element with a sequence.

In [10]:
net.run(sess, np.array([vec('A' * 20) for x in range(20)]))

array([[[0.25172085],
        [0.25115234],
        [0.2522114 ],
        [0.24931656],
        [0.24883813],
        [0.24800994],
        [0.24779543],
        [0.24757418],
        [0.24749605],
        [0.24743049],
        [0.24740025],
        [0.24737744],
        [0.2473646 ],
        [0.24735531],
        [0.24734937],
        [0.24734512],
        [0.24734223],
        [0.2473401 ],
        [0.24733864],
        [0.24733754]],

       [[0.25172085],
        [0.25115234],
        [0.2522114 ],
        [0.24931656],
        [0.24883813],
        [0.24800994],
        [0.24779543],
        [0.24757418],
        [0.24749605],
        [0.24743049],
        [0.24740025],
        [0.24737744],
        [0.2473646 ],
        [0.24735531],
        [0.24734937],
        [0.24734512],
        [0.24734223],
        [0.2473401 ],
        [0.24733864],
        [0.24733754]],

       [[0.25172085],
        [0.25115234],
        [0.2522114 ],
        [0.24931656],
        [0.24883813],
      